Comentarios obtenidos por las IAs y Stack Over

 - To detect cars in a video, you will need to use the yolo predict command with the model parameter set to yolov8.pt
 - Para diferenciar los tipos de vehículos, deberás agregar una capa adicional al modelo. Esta capa adicional se llama clasificador.


Preguntas personales
- Como funciona el modelo


To investigate:
PyTorch / CUDA / cuDNN what are and for wich porpuse works...
Que es SOTA : Ultralytics YOLOv8 is a cutting-edge, state-of-the-art (SOTA)

Code example :

https://colab.research.google.com/github/ultralytics/ultralytics/blob/main/examples/tutorial.ipynb#scrollTo=bpF9-vS_DAaf


https://colab.research.google.com/github/roboflow-ai/notebooks/blob/main/notebooks/how-to-track-and-count-vehicles-with-yolov8.ipynb?ref=blog.roboflow.com#scrollTo=c4okzdHlKMaj

*** About Utilities ***
If you want to specify which device to use for the model, you can pass the argument device followed by the number of the device that you want to use. For example, device=0


In [13]:
!nvidia-smi #monitorear cómo se está utilizando tu GPU

Sat Aug  5 15:37:04 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 531.79                 Driver Version: 531.79       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3070       WDDM | 00000000:07:00.0  On |                  N/A |
|  0%   53C    P8               20W / 240W|   1201MiB /  8192MiB |     25%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [15]:
import os  # Importa la biblioteca 'os' que permite interactuar con el sistema operativo.

# La función os.getcwd() obtiene el directorio de trabajo actual, es decir, la ubicación en tu sistema de archivos
# donde se está ejecutando tu script de Python. 
# Guarda el resultado en la variable 'HOME'.
HOME = os.getcwd()  

# Imprime el contenido de la variable 'HOME', que es el directorio de trabajo actual.
print(HOME)

c:\Users\Edgar\OneDrive\Escritorio\Data_and_ML\Car_Detection


In [ ]:
# %cd es un comando mágico en Jupyter Notebook que cambia el directorio de trabajo.
# {HOME} es una variable que contiene la ruta al directorio de trabajo que obtuviste con os.getcwd().
# Por lo tanto, este comando cambia el directorio de trabajo al valor de {HOME}.
%cd {HOME}


In [ ]:
"""Esa línea de código es un comando shell específicamente diseñado para descargar archivos de Google Drive desde la línea de comandos.
"""
# Usa wget para descargar un archivo de Google Drive. Necesitas un código de confirmación para descargar archivos grandes,
# por lo que primero usa wget para obtener la página de descarga y extrae el código de confirmación con sed.
# Luego, vuelve a usar wget para descargar el archivo con el código de confirmación.
# Finalmente, elimina el archivo de cookies.


#!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-" -O vehicle-counting.mp4 && rm -rf /tmp/cookies.txt


Subiendo los videos con los cuales se va a trabajar...

In [45]:
import cv2 
#video = cv2.VideoCapture('C:/Users/Edgar/OneDrive/Escritorio/Data_and_ML/Car_Detection/Videos/01/00-02-07_2a.mkv')
# En caso de que queramos usar una cadena de texto cruda, para no tener problemas con los \ podemos ingresarlo de la siguiente manera
video = cv2.VideoCapture(r'C:\Users\Edgar\OneDrive\Escritorio\Data_and_ML\Car_Detection\Videos\01\00-02-07_2a.mkv') # <-- Cargamos el video
# Obtiene los frames por segundo (fps) del video
fps = video.get(cv2.CAP_PROP_FPS)

# Calcula cuántos frames necesitas leer para obtener 10 segundos de video
frames_for_10_seconds = int(60 * fps)

# Define el tamaño de la fuente y el grosor
font_scale = 1
thickness = 2

# Lee y muestra los frames uno por uno
for i in range(frames_for_10_seconds):
    # Lee el próximo frame
    ret, frame = video.read()

    # Si no se pudo leer el frame, rompe el ciclo
    if not ret:
        break

    # Obtiene el tamaño del texto
    text_size = cv2.getTextSize(f'FPS: {fps}', cv2.FONT_HERSHEY_SIMPLEX, font_scale, thickness)[0]

    # Calcula la posición x para que el texto esté alineado a la derecha
    x = frame.shape[1] - text_size[0] - 10  # 10 pixels de margen desde el borde derecho

    # Escribe los fps en el frame
    cv2.putText(frame, f'FPS: {fps}', (x, 30), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255, 255, 255), thickness)

    # Muestra el frame en una ventana llamada 'video'
    cv2.imshow('video', frame)

    # Espera 1000 / fps milisegundos antes de mostrar el siguiente frame,
    # para que la velocidad de reproducción sea la correcta
    if cv2.waitKey(int(1000 / fps)) & 0xFF == ord('q'):
        break

# Cuando termines, libera el objeto video y destruye todas las ventanas creadas por cv2
video.release()
cv2.destroyAllWindows()

En el siguiente codigo, aplicamos para control de direccion algunos principios

In [49]:
# En caso de que queramos usar una cadena de texto cruda, para no tener problemas con los \ podemos ingresarlo de la siguiente manera
video = cv2.VideoCapture(r'C:\Users\Edgar\OneDrive\Escritorio\Data_and_ML\Car_Detection\Videos\01\00-02-07_2a.mkv') # <-- Cargamos el video
# Obtiene los frames por segundo (fps) del video
fps = video.get(cv2.CAP_PROP_FPS)
# Obtiene los frames por segundo (fps) del video
fps = video.get(cv2.CAP_PROP_FPS)

# Define el tamaño de la fuente y el grosor
font_scale = 1
thickness = 2

# Factor de salto de frames para adelanto y retroceso
skip_frames = 30  # Cambia esto al número de frames que quieras saltarte a la vez

# Lee y muestra los frames uno por uno
i = 0
while True:
    # Lee el próximo frame
    ret, frame = video.read()

    # Si no se pudo leer el frame, rompe el ciclo
    if not ret:
        break

    # Obtiene el tamaño del texto
    text_size = cv2.getTextSize(f'FPS: {fps}', cv2.FONT_HERSHEY_SIMPLEX, font_scale, thickness)[0]

    # Calcula la posición x para que el texto esté alineado a la derecha
    x = frame.shape[1] - text_size[0] - 100  # 100 pixels de margen desde el borde derecho

    # Escribe los fps en el frame
    cv2.putText(frame, f'FPS: {fps}', (x, 30), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (150, 250, 30), thickness)

    # Muestra el frame en una ventana llamada 'video'
    cv2.imshow('video', frame)

    # Espera 1000 / fps milisegundos antes de mostrar el siguiente frame,
    # para que la velocidad de reproducción sea la correcta
    key = cv2.waitKey(int(1000 / fps)) & 0xFF

    # Si se presiona la tecla 'q', rompe el ciclo
    if key == ord('q'):
        break

    # Si se presiona la tecla 'd', adelanta el video
    elif key == ord('d'):
        for _ in range(skip_frames):
            video.read()
            i += 1

    # Si se presiona la tecla 'a', retrocede el video
    elif key == ord('a'):
        i = max(0, i - skip_frames)
        video.set(cv2.CAP_PROP_POS_FRAMES, i)
        
    i += 1

# Cuando termines, libera el objeto video y destruye todas las ventanas creadas por cv2
video.release()
cv2.destroyAllWindows()

In [19]:
# !yolo predict model=yolov8n.pt source='https://ultralytics.com/images/zidane.jpg' <== Esta madre significa ejecutar en consola

from ultralytics import YOLO  # Importa la clase YOLO de la biblioteca Ultralytics.
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Creación de una nueva instancia de YOLO
model = YOLO('yolov8n.yaml')  # Construye un nuevo modelo desde cero con la arquitectura especificada en 'yolov8n.yaml'.
model = YOLO('yolov8n.pt')  # Carga un modelo pre-entrenado ('yolov8n.pt'). Recomendado para entrenamientos.

# Uso del modelo
results = model.train(data='coco128.yaml', epochs=3)  # Entrena el modelo con los datos especificados en 'coco128.yaml' durante 3 epochs. <- pasada completa a través de todo el conjunto de datos
results = model.val()  # Evalúa el rendimiento del modelo en el conjunto de datos de validación.
results = model('https://ultralytics.com/images/bus.jpg')  # Realiza una predicción en una imagen ubicada en la URL especificada.
results = model.export(format='onnx')  # Exporta el modelo al formato ONNX, para uso en diferentes frameworks de aprendizaje profundo.


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

============== Diagnostic Run torch.onnx.export version 2.0.1+cpu ==============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [53]:
# Cambia el directorio de trabajo actual al directorio especificado por la variable HOME
#%cd {HOME}

# Clona el repositorio de GitHub ByteTrack al directorio de trabajo actual
#!git clone https://github.com/ifzhang/ByteTrack.git

# Cambia el directorio de trabajo actual a la carpeta ByteTrack
%cd {HOME}/ByteTrack

# Usa sed para editar el archivo requirements.txt, cambiando onnx==1.8.1 a onnx==1.9.0
# Este es un ajuste de compatibilidad para resolver un problema

#!sed -i 's/onnx==1.8.1/onnx==1.9.0/g' requirements.txt <-- Este comando no lo reconoce, que no es un lenguaje propio es un bash o algo asi, investigar de nuevo
with open('requirements.txt', 'r') as file:
    data = file.read()

data = data.replace('onnx==1.8.1', 'onnx==1.9.0')

with open('requirements.txt', 'w') as file:
    file.write(data)
# Instala las dependencias especificadas en el archivo requirements.txt
!pip3 install -q -r requirements.txt

# Ejecuta el script setup.py con la opción "develop"
# Esto instala el proyecto en modo de desarrollo
!python3 setup.py -q develop

# Instala el paquete cython_bbox
%pip install -q cython_bbox

# Instala el paquete onemetric
%pip install -q onemetric

# Instala los paquetes loguru, lap y thop
%pip install -q loguru lap thop

# Importa el módulo display desde IPython
from IPython import display

# Limpia la salida del Jupyter Notebook
display.clear_output()

# Importa el módulo sys
import sys

# Añade la ruta a la carpeta ByteTrack a las rutas donde Python buscará módulos para importar
sys.path.append(f"{HOME}/ByteTrack")

# Importa el módulo yolox
import yolox

# Imprime la versión de YOLOX
print("yolox.__version__:", yolox.__version__)

#Se necesitaba instalar modulo thop
#paquete es útil para hacer comparaciones entre diferentes modelos de redes neuronales 

ModuleNotFoundError: No module named 'thop'